**Inspired from - https://www.tensorflow.org/tutorials/generative/pix2pix**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import datetime
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import csv
import numpy as np
import os
import random
import time
import  seaborn as sns
import tifffile as tiff
import matplotlib
matplotlib.use('Agg')
import glob
from matplotlib import pyplot as plt
from PIL import Image
import cv2
from skimage.transform import resize
from skimage.morphology import binary_opening, disk, skeletonize
from skimage.measure import label, regionprops
import pandas as pd
from skimage.filters import threshold_otsu
try:
    from skimage.io import imsave as imwrite
    from skimage.io import imread
except ImportError:
    from cv2 import imread
    from cv2 import imwrite

from sklearn.model_selection import train_test_split

In [ ]:
!pip install -q -U tensorboard

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
import csv
import numpy as np
import os
import random
import time
import  seaborn as sns
import tifffile as tiff
import matplotlib
matplotlib.use('Agg')
import glob
from matplotlib import pyplot as plt
from PIL import Image
import cv2
from skimage.transform import resize
from skimage.morphology import binary_opening, disk, skeletonize
from skimage.measure import label, regionprops
import pandas as pd
from skimage.filters import threshold_otsu
try:
    from skimage.io import imsave as imwrite
    from skimage.io import imread
except ImportError:
    from cv2 import imread
    from cv2 import imwrite

from sklearn.model_selection import train_test_split

class KDatasetPreparation():
    def __init__(self,train_csv,
                      train_img_folder,
                      resize_factor = 1,
                      point_sampling_level = 2,
                      superImposed = True,
                      patch_size = 512, #read_size
                      stride = 512,
                      save_gan_pair_list = True,
                      save_patch = True,
                      save_folder=None):
        self.train_df = pd.read_csv(train_csv)
        self.resize_factor = resize_factor
        self.save_folder =save_folder
        self.point_sampling_level =point_sampling_level
        self.train_img_folder = train_img_folder
        self.superImposed = superImposed
        self.patch_size = patch_size
        self.stride = stride
        self.save_patch = save_patch
        self.kernel = disk(2)
        if self.save_folder is None:
            self.save_folder = './tmp'
        self.gan_pair_list = []
        self.wsi_list = []
        self.save_img_mask =  os.path.join(self.save_folder,'imgdata')
        self.save_train_img =  os.path.join(self.save_folder,'patch')
        self.save_train_mask =  os.path.join(self.save_folder,'mask')
        os.makedirs(self.save_folder,exist_ok=True)
        os.makedirs(self.save_img_mask,exist_ok=True)
        os.makedirs(self.save_train_img,exist_ok=True)
        os.makedirs(self.save_train_mask,exist_ok=True)
        
        self.save_gan_pair_list = save_gan_pair_list

        print(self.train_df.head())


    def get_mask_from_submission(self,subfilepath,testfolder,tryboth=True):
        '''
        test mask and rle
        '''
        sub_df = pd.read_csv(subfilepath)
        for idx,file in enumerate(sub_df['id']):
            ts = time.time()
            print('Processingg...',file)
            filepath = os.path.join(testfolder,file+'.tiff')
            if not os.path.exists(filepath):
                continue
            wsiImage = tiff.imread(filepath)
            if len(wsiImage.shape) == 5:
                wsiImage = np.transpose(wsiImage.squeeze(), (1, 2, 0))

            if wsiImage.shape[0]==3:
                wsiImage = np.transpose(wsiImage, (1, 2, 0))

            wsiMask1 = rle2mask(sub_df['predicted'][idx],
                                 (wsiImage.shape[1], wsiImage.shape[0]))

            filepath = os.path.join(testfolder, file + '_mask.tiff')
            print('time taken',time.time()-ts)
            cv2.imwrite(filepath, wsiMask1)
            if tryboth:
                #rle_encode_less_memory
                h1,w1 = wsiMask1.shape
                maskhalf = cv2.resize(wsiMask1,(w1,h1))#interpolation=cv2.INTER_AREA)
                tsb = time.time()
                code1 = rle_encode_less_memory(maskhalf,img_size_hw=(h1,w1))
                print('time taken-les memory-', time.time() - tsb)



                tsb = time.time()
                code2 = rle_numba_encode_method2(maskhalf, img_size_hw=(h1, w1))
                print('time taken-2', time.time() - tsb)

                print('if code is same-numba',code1==code2)


                if 1:
                    mask1_re = rle2mask(code1,(w1,h1))
                    filepath = os.path.join(testfolder, file + '_mask_re1.tiff')
                    cv2.imwrite(filepath, mask1_re)
                    mask2_re = rle2mask(code2,(w1,h1))
                    filepath = os.path.join(testfolder, file + '_mask_re2.tiff')
                    cv2.imwrite(filepath, mask2_re)
                    print('are mask same',np.array_equal(mask1_re,mask2_re))



    def save_train_imgAndMask(self):
        for idx,file in enumerate(self.train_df['id']):
            print('Processingg...',file)
            filepath = os.path.join(self.train_img_folder,file+'.tiff')
            wsiImage = tiff.imread(filepath)

            if len(wsiImage.shape) == 5:
                wsiImage = np.transpose(wsiImage.squeeze(), (1, 2, 0))

            if wsiImage.shape[0]==3:
                wsiImage = np.transpose(wsiImage, (1, 2, 0))

            wsiMask = rle2mask(self.train_df['encoding'][idx],
                                 (wsiImage.shape[1], wsiImage.shape[0]))
            wsiMask = wsiMask*255

            print(f'Image Shape: {wsiImage.shape}')
            print(f'Mask Shape: {wsiMask.shape}')

            if True:
                filepath = os.path.join(self.save_img_mask, file + f'_mask.tiff')
                cv2.imwrite(filepath, wsiMask)
            if self.point_sampling_level !=1:
                factor = 2**self.point_sampling_level
                rescaled = (wsiImage.shape[1] // factor, wsiImage.shape[0] // factor)
                wsiImage = cv2.resize(wsiImage, rescaled,interpolation=cv2.INTER_LINEAR)
                wsiMask = cv2.resize(wsiMask, rescaled,interpolation=cv2.INTER_AREA)

                filepath = os.path.join(self.save_img_mask,file+f'_resizefactor_{self.point_sampling_level}.tiff')
                cv2.imwrite(filepath,wsiImage)
                filepath = os.path.join(self.save_img_mask,file+f'_resizefactor_{self.point_sampling_level}_mask.tiff')
                cv2.imwrite(filepath, wsiMask)
                if self.superImposed:
                    vis_img = wsiImage
                    vis_img[wsiMask>0, 0] = 0
                    vis_img[wsiMask>0, 1] = 255
                    vis_img[wsiMask>0, 2] = 0
                    filepath = os.path.join(self.save_img_mask,
                                            file + f'_resizefactor_{self.point_sampling_level}_sumperImposed.tiff')
                    cv2.imwrite(filepath, vis_img)

    def get_tissue_mask(self,img,method='std',stdthd=5):

        if method=='std':
            mask = np.std(img,axis=-1) >stdthd
        else:
            #otsu
            if np.ndim(img) == 3:
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            thd = threshold_otsu(img)
            mask = img < thd

        mask = self.postprocessing(mask)
        return mask

    def process_patch_locations(self,imgName,idx):
        self.wsi_list.append(imgName)
        filepath = os.path.join(self.train_img_folder, imgName + '.tiff')
        wsiImage = tiff.imread(filepath)
        if len(wsiImage.shape) == 5:
            wsiImage = np.transpose(wsiImage.squeeze(), (1, 2, 0))
        if wsiImage.shape[0] == 3:
            wsiImage = np.transpose(wsiImage, (1, 2, 0))

        wsiMask = rle2mask(self.train_df['encoding'][idx],
                           (wsiImage.shape[1], wsiImage.shape[0]))
        wsiMask = wsiMask * 255

        factor = 2 ** self.point_sampling_level
        rescaled = (wsiImage.shape[1] // factor, wsiImage.shape[0] // factor)
        wsiImage_rsz = cv2.resize(wsiImage, rescaled, interpolation=cv2.INTER_LINEAR)
        #wsiImage_gray = cv2.cvtColor(wsiImage_rsz,cv2.COLOR_RGB2GRAY)
        valid_region_mask = self.get_tissue_mask(wsiImage_rsz,method='std')

        H,W = valid_region_mask.shape

        if False:
            mask_filename = os.path.join(self.save_img_mask, f"{imgName}_tissuemask.png")
            tiff.imwrite(mask_filename, 255*np.uint8(valid_region_mask))

        stride_mask_level = self.stride // factor
        patch_mask_level = self.patch_size // factor

        grid_mask = np.zeros_like(valid_region_mask)
        x_grid = np.arange(0, W, stride_mask_level)
        y_grid = np.arange(0, H, stride_mask_level)
        Xcord, Ycord = np.meshgrid(x_grid, y_grid)
        grid_mask[Ycord, Xcord] = True
        yxPair = np.argwhere(grid_mask)
        area_thd = 0.20*patch_mask_level*patch_mask_level
        yxpairList=[]
        for y, x in yxPair:
            if np.sum(valid_region_mask[y:y + patch_mask_level, x: x + patch_mask_level]) > area_thd:
                # if np.sum(valid_region_mask[y:y + patch_mask_level, x: x + patch_mask_level]==0) > 8*area_thd:
                #     continue
                imgPatch = wsiImage[y*factor:y*factor+self.patch_size,x*factor:x*factor+self.patch_size,:]
                if np.std(imgPatch) < 10:
                    continue
                if len(np.unique(imgPatch)) <25:
                    continue
                yxpairList.append((imgName,y * factor, x * factor))

        if self.save_patch:
            print('Saving patches...')
            for item in yxpairList:
                flag = 0
                imagename,y,x = item
                img_crop = wsiImage[y:y+self.patch_size,x:x+self.patch_size,:]
                mask_crop = wsiMask[y:y+self.patch_size,x:x+self.patch_size]
                mask_crop = cv2.dilate(mask_crop, disk(2), iterations=1)
                if np.sum(mask_crop)>0:
                    flag = 1
                mask_deeplab = np.zeros_like(mask_crop)
                mask_deeplab[mask_crop==255] = 1
                mask_dilated = cv2.dilate(mask_crop,self.kernel,iterations=2)
                mask_boundray = mask_dilated - mask_crop
                mask_deeplab[mask_boundray>0]=255
                patch_filename = os.path.join(self.save_train_img,f"{imagename}_Y_{y}_X_{x}_tn_{flag}.png")
                mask_filename = os.path.join(self.save_train_mask,f"{imagename}_Y_{y}_X_{x}_tn_{flag}.png")
                cv2.imwrite(patch_filename,cv2.cvtColor(img_crop,cv2.COLOR_RGB2BGR))
                cv2.imwrite(mask_filename,mask_deeplab)#cv2.COLOR_RGB2BGR))

                
        
        if self.save_gan_pair_list:
            print('Gan pairpatches...')
            for item in yxpairList:
                flag = 0
                imagename,y,x = item
                img_crop = wsiImage[y:y+self.patch_size,x:x+self.patch_size,:]
                mask_crop = wsiMask[y:y+self.patch_size,x:x+self.patch_size]
                mask_crop = cv2.dilate(mask_crop, disk(2), iterations=1)
                if np.sum(mask_crop)>0:
                    flag = 1
                mask_deeplab = np.zeros_like(mask_crop)
                mask_deeplab[mask_crop==255] = 1
                mask_dilated = cv2.dilate(mask_crop,self.kernel,iterations=2)
                mask_boundray = mask_dilated - mask_crop
                mask_deeplab[mask_boundray>0]=255
                patch_filename = os.path.join(self.save_train_img,f"{imagename}_Y_{y}_X_{x}_tn_{flag}.png")
                mask_filename = os.path.join(self.save_train_mask,f"{imagename}_Y_{y}_X_{x}_tn_{flag}.png")
                
                if np.sum(mask_crop >0) > 0.2*mask_crop.shape[0]*mask_crop.shape[1]:
                    self.gan_pair_list.append([patch_filename,mask_filename])
                    cv2.imwrite(patch_filename,cv2.cvtColor(img_crop,cv2.COLOR_RGB2BGR))
                    cv2.imwrite(mask_filename,mask_deeplab)#cv2.COLOR_RGB2BGR))

        return yxpairList

    def postprocessing(self,img):
        if img.dtype == np.bool:
            img = 255* np.uint8(img)
        kernel = disk(2)
        # remove unwanted small detection
        # self.stitch_image = cv2.erode(self.stitch_image,kernel,iterations=1)
        # remove small white object- opening  ; Closing fills the hole in white image
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=2)
        return img

    def save_train_locations(self):
        yxLocations = []
        for idx, file in enumerate(self.train_df['id']):
            print('Processingg...', file)
            yxLocations += self.process_patch_locations(file,idx)


        np.save(os.path.join(self.save_folder,'yxLocations.npy'),yxLocations)



    def split_train_valid(self,patch_folder=None,folds=None,test_ratio=0.25):
        if patch_folder is None:
            patch_folder = self.save_train_img
        files = list(glob.glob(os.path.join(patch_folder, '*.png')))

        suffix = 'kd_625_'
        txtfolderpath = os.path.dirname(patch_folder)
        if folds is None:
            imageNameList, labelNameList = [], []
            for file in files:
                imageNameList.append(file)
                lablelstr = file.split('.')[0]
                label = int(lablelstr.split('_')[-1])
                labelNameList.append(label)

            X_train, X_test, y_train, y_test = train_test_split(imageNameList, labelNameList,
                                                                test_size=test_ratio, stratify=labelNameList,
                                                                shuffle=True, random_state=42)

            with open(os.path.join(txtfolderpath, f'train_{suffix}.txt'), 'w') as fp:
                for file in X_train:
                    basename = os.path.basename(file)
                    base = basename.split('.')[0]
                    fp.write(base)
                    fp.write('\n')

            with open(os.path.join(txtfolderpath, f'val_{suffix}.txt'), 'w') as fp:
                for file in X_test:
                    basename = os.path.basename(file)
                    base = basename.split('.')[0]
                    fp.write(base)
                    fp.write('\n')

        else:
            total_wsi  = len(self.wsi_list)
            print(self.wsi_list)
            for key in folds:
                valid_files = folds[key]
                X_train ,X_test = [] ,[]
                for file in files:
                    basename = os.path.basename(file)
                    first_name = basename.split('_')[0]
                    if first_name in valid_files:
                        X_test.append(file)
                    else:
                        X_train.append(file)


                with open(os.path.join(txtfolderpath, f'train_{suffix}_{key}.txt'), 'w') as fp:
                    for file in X_train:
                        basename = os.path.basename(file)
                        base = basename.split('.')[0]
                        fp.write(base)
                        fp.write('\n')

                with open(os.path.join(txtfolderpath, f'val_{suffix}_{key}.txt'), 'w') as fp:
                    for file in X_test:
                        basename = os.path.basename(file)
                        base = basename.split('.')[0]
                        fp.write(base)
                        fp.write('\n')

        print('Files are written at-',txtfolderpath)

In [ ]:
# !rm -r ./imgdata
# !rm -r ./mask
# !rm -r ./patch

In [ ]:

import PIL
import sys
PIL.Image.MAX_IMAGE_PIXELS = 933120000
ts  = time.time()
ROOT_PATH = r'/kaggle/input/hubmap-kidney-segmentation/'
TRAIN_CSV = os.path.join(ROOT_PATH,'train.csv')
TRAIN_IMAGE_FOLDER = os.path.join(ROOT_PATH,'train')
SAVE_FOLDER = r'./'
GAN_LIST = True

FOLDS_WSI_WISE = {
         'fold0': ['2f6ecfcdf','aaa6a05cc'],
         'fold1': ['cb2d976f4', '0486052bb'],
         'fold2': ['e79de561c', '095bf7a1f','1e2425f28'],
         }

ts = time.time()
objData= KDatasetPreparation(train_csv=TRAIN_CSV,
                             point_sampling_level=3,
                             save_patch=False,
                             save_gan_pair_list = GAN_LIST,
                             patch_size=625,# idear i sto have big an dthen crop 512
                             save_folder=SAVE_FOLDER,
                             train_img_folder=TRAIN_IMAGE_FOLDER)
# objData.get_mask_from_submission(subfilepath=r'C:\Users\sjha157731\Downloads\test\submission.csv',
#                                  testfolder=r'C:\Users\sjha157731\Downloads\test')

#objData.save_train_imgAndMask()
#objData.save_train_locations()
#np.save(os.path.join(SAVE_FOLDER,'gan_pair_list.npy'),objData.gan_pair_list)
#objData.split_train_valid(test_ratio=0.25,folds=FOLDS_WSI_WISE)
#print('Total Time for pair genertaion',time.time()-ts)



In [ ]:
import shutil
patch_path = r'/kaggle/input/pix2pixmaskimage/img_patches/patch'
mask_path = r'/kaggle/input/pix2pixmaskimage/mask_patches/mask'
npy_path = r'/kaggle/input/pix2pixmaskimage/gan_pair_list.npy'
#dst = r'./mask'
#shutil.copytree(mask_path,dst)
#dst = r'./patch'
#shutil.copytree(patch_path,dst)
#shutil.copy(npy_path,'./')

In [ ]:
#!zip -r ./img_patches.zip ./patch
#!zip -r ./mask_patches.zip ./mask

In [ ]:
import numpy as np
#sanity chekc
pathdata = r'/kaggle/input/pix2pixmaskimage/gan_pair_list.npy'
data = np.load(pathdata,allow_pickle=True)
data = np.array(data)
print(data.shape)

#updated path
updated = []
for i in range(data.shape[0]):
    item = data[i]
    x_c = item[0].replace('./','/kaggle/input/pix2pixmaskimage/img_patches/')
    y_c = item[1].replace('./','/kaggle/input/pix2pixmaskimage/mask_patches/')
    updated.append([x_c,y_c])
updated = np.array(updated)
np.save('./gan_pair_list_updated.npy',updated)

**PIX2PIX GAN - Define model **

In [ ]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import time
'''
#https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/pix2pix.ipynb#scrollTo=dIbRPFzjmV85
#https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/pix2pix.ipynb
#https://machinelearningmastery.com/what-are-generative-adversarial-networks-gans/
#define models
#Genereator - a Unet - Discriminator - patchGan
#Kernel size = 4, stride =2
Taken from paper-
    Generator - UNET-
        *Encoder:
            C64-C128-C256-C512-C512-C512-C512-C512
        *Decoder:
            CD512-CD512-CD512-C512-C256-C128-C64
        *   (D/E)Let Ck denote a Convolution-BatchNorm-ReLU layer with k filters. CDk denotes a 
            Convolution-BatchNorm-Dropout-ReLU layer with a dropout rate of 50%. 
        *   (D/E)All convolutions are 4x4 spatial filters applied with stride 2. 
            Convolutions in the encoder, and in the discriminator, downsample
            by a factor of 2, whereas in the decoder they upsample by a
            factor of 2.
        *   (E)All ReLUs in the encoder are leaky, with slope 0.2, 
        *   (E)ReLUs in the decoder are not leaky.
        *   (D/E)Weights were initialized from a Gaussian distribution with mean 0 and standard deviation 0.02
        *   (E)Batch-Norm is not applied to the first C64 layer in the encoder
        *   (D)After the last layer in the decoder, a convolution is applied to map to the number of output channels (3 in general,
            except in colorization, where it is 2), followed by a Tanh function
    
    Discriminator- patchGAN
         *  The 70 x70 discriminator architecture is:  C64-C128-C256-C512
            After the last layer, a convolution is applied to map to
            a 1-dimensional output, followed by a Sigmoid function.
        *   As an exception to the above notation, BatchNorm is not
            applied to the first C64 layer. All ReLUs are leaky, with
            slope 0.2
'''
def DownsampleBlock(nFilter,kernel=4,stride=2,apply_batchnorm=True,apply_dropout=False):
    #Conv->BN->Leaky-Relu
    initializer = tf.random_normal_initializer(0., 0.02)
    block = tf.keras.Sequential()
    block.add(tf.keras.layers.Conv2D(filters=nFilter,kernel_size=kernel,strides=stride,
                                     padding='same',kernel_initializer=initializer,
                                     use_bias=False))

    if apply_batchnorm:
        block.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        block.add(tf.keras.layers.Dropout(alpha=0.5))

    block.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    return block

def UpsampleBlock(nFilter, kernel=4, stride=2, apply_batchnorm=True, apply_dropout=False):
    # Conv->BN->Dropout->Relu
    initializer = tf.random_normal_initializer(0., 0.02)
    block = tf.keras.Sequential()
    block.add(tf.keras.layers.Conv2DTranspose(filters=nFilter, kernel_size=kernel, strides=stride,
                                     padding='same', kernel_initializer=initializer,
                                     use_bias=False))

    if apply_batchnorm:
        block.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        block.add(tf.keras.layers.Dropout(rate=0.5))

    block.add(tf.keras.layers.ReLU())

    return block


def generator_model(input_shape=[256,256,3]):
    nFilters = 64
    nUpFilters = 512 #(64*8)
    OUTPUT_CHANNELS = input_shape[2]
    input = tf.keras.layers.Input(shape=input_shape)

    #encoder -C64-C128-C256-C512-C512-C512-C512-C512
    c064 = DownsampleBlock(nFilter=nFilters,apply_batchnorm=False,apply_dropout=False)(input) #-1,128x128x64
    c128 = DownsampleBlock(nFilter=nFilters*2,apply_dropout=False)(c064)   #-1,64x64x128
    c256 = DownsampleBlock(nFilter=nFilters*4,apply_dropout=False)(c128)   #-1,32x32x256
    c0x512 = DownsampleBlock(nFilter=nFilters*8,apply_dropout=False)(c256)   #-1,16x16x512
    c1x512 = DownsampleBlock(nFilter=nFilters*8,apply_dropout=False)(c0x512)   #-1,8x8x512
    c2x512 = DownsampleBlock(nFilter=nFilters*8,apply_dropout=False)(c1x512)   #-1,4x4x512
    c3x512 = DownsampleBlock(nFilter=nFilters*8,apply_dropout=False)(c2x512)   #-1,2x2x512
    c4x512 = DownsampleBlock(nFilter=nFilters*8,apply_dropout=False)(c3x512)   #-1,1x1x512

    #decoder - CD512-CD512-CD512-C512-C256-C128-C64
    u0512 = UpsampleBlock(nFilter=nUpFilters,apply_dropout=True)(c4x512) #-1,2x2x512
    cat_1 = tf.keras.layers.Concatenate()([u0512, c3x512])               #-1,2x2x2048
    u1512 = UpsampleBlock(nFilter=nUpFilters,apply_dropout=True)(cat_1)  #-1,4x4x1024
    cat_2 = tf.keras.layers.Concatenate()([u1512, c2x512])               #-1,4x4x2048
    u2512 = UpsampleBlock(nFilter=nUpFilters, apply_dropout=True)(cat_2) # -1,8x8x1024
    cat_3 = tf.keras.layers.Concatenate()([u2512, c1x512])               # -1,8x8x2048
    u3512 = UpsampleBlock(nFilter=nUpFilters, apply_dropout=False)(cat_3)  # -1,16x16x1024
    cat_4 = tf.keras.layers.Concatenate()([u3512, c0x512])                 # -1,16x16x2048

    u4512 = UpsampleBlock(nFilter=nUpFilters//2, apply_dropout=False)(cat_4)   # -1,32x32x256
    cat_5 = tf.keras.layers.Concatenate()([u4512, c256])                       # -1,32x32x512

    u5512 = UpsampleBlock(nFilter=nUpFilters // 4, apply_dropout=False)(cat_5)  # -1,64x64x128
    cat_6 = tf.keras.layers.Concatenate()([u5512, c128])                        # -1,64x64x256

    u6512 = UpsampleBlock(nFilter=nUpFilters // 8, apply_dropout=False)(cat_6)  # -1,128x128x64
    cat_7 = tf.keras.layers.Concatenate()([u6512, c064])                        #-1,128x128x128

    #Exception for last layer
    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                           strides=2,
                                           padding='same',
                                           kernel_initializer=initializer,
                                           activation='tanh') (cat_7)           # -1x256x256x3

    model = tf.keras.Model(inputs=input, outputs=last)
    return model

def discriminator_model(input_shape=[256,256,3],outlogit_no_sigmoid=True):
    '''
    This model takes 2 inouts - src & target
    Each 30x30 patch of the output classifies a 70x70 portion of the
    input image (such an architecture is called a PatchGAN).
    '''
    initializer = tf.random_normal_initializer(0., 0.02)
    inp = tf.keras.layers.Input(shape=input_shape, name='input_image')
    tar = tf.keras.layers.Input(shape=input_shape, name='target_image')

    catx = tf.keras.layers.concatenate([inp, tar])  # (-1x256x256xchannels*2)
    x = DownsampleBlock(nFilter=64,apply_batchnorm=False,apply_dropout=False)(catx) #-1x128x128x64
    x = DownsampleBlock(nFilter=128,apply_dropout=False)(x)     #-1x64x64x128
    x = DownsampleBlock(nFilter=256,apply_dropout=False)(x)     #-1x32x32x256

    #taken from Tensorflow Colab
    pad = tf.keras.layers.ZeroPadding2D()(x)            #-1x34x34x256
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(pad)  #-1x31x31x512

    bn = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)(bn)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  #-1x33x33x512

    #as per paper
    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                  kernel_initializer=initializer)(zero_pad2)  #-1x30x30x1
    if not outlogit_no_sigmoid:
        last = tf.keras.layers.Activation(activation='sigmoid')(last)

    return tf.keras.Model(inputs=[inp, tar], outputs=last)


In [ ]:
def load_image_train(image_file):
    input_image, real_image = load_local(image_file)
    input_image, real_image = random_jitter(input_image, real_image)
    input_image, real_image = normalize(input_image, real_image)
    return input_image, real_image

def load_image_test(image_file):
    input_image, real_image = load_local(image_file)
    input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
    input_image, real_image = normalize(input_image, real_image)
    return input_image, real_image

def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)

    w = tf.shape(image)[1]

    w = w // 2
    real_image = image[:, :w, :]
    input_image = image[:, w:, :]

    input_image = tf.cast(input_image, tf.float32)
    real_image = tf.cast(real_image, tf.float32)

    return input_image, real_image

def load_local(image_file):
    #0: Orig image /1:- Mask
    image = tf.io.read_file(image_file[0])
    image = tf.image.decode_png(image)
    real_image = tf.cast(image, tf.float32)
    image = tf.io.read_file(image_file[1])
    image = tf.image.decode_png(image)
    #image = tf.print(image,[image.shape])
    if True:
          op1 = tf.zeros_like(image)  # 0
          op2 = op1 + 255  # 255
          image = tf.where(tf.equal(image, tf.constant(255, dtype=tf.uint8)), op1, image)
          #image = tf.where(tf.equal(image, tf.constant(1, dtype=tf.uint8)), op2, label)
    image = tf.image.grayscale_to_rgb(image)
    input_image = tf.cast(image, tf.float32)

    return input_image, real_image

def resize(input_image, real_image, height, width):
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, real_image


def random_crop(input_image, real_image):
    stacked_image = tf.stack([input_image, real_image], axis=0)
    cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image[0], cropped_image[1]
# normalizing the images to [-1, 1]

def normalize(input_image, real_image):
    #input_image = (input_image / 127.5) - 1 # already -0/1 range
    real_image = (real_image / 127.5) - 1

    return input_image, real_image

@tf.function()
def random_jitter(input_image, real_image):
    # resizing to 286 x 286 x 3
    input_image, real_image = resize(input_image, real_image, 286, 286)

    # randomly cropping to 256 x 256 x 3
    input_image, real_image = random_crop(input_image, real_image)

    if tf.random.uniform(()) > 0.5:
        # random mirroring
        input_image = tf.image.flip_left_right(input_image)
        real_image = tf.image.flip_left_right(real_image)

    return input_image, real_image

In [ ]:
def generator_loss(discriminator_out_gen_in,generator_out,target):
    '''
    Idea is that discriminator shld predict real for photo generated
    from generator for real training data
    :param discriminator_out_gen_in:
    :param generator_out:
    :param target:
    '''
    LAMBDA = 100
    #to maintain structure
    gen_L1_loss = tf.reduce_mean(tf.abs(target - generator_out))
    #bce loss-
    gt_label = tf.ones_like(discriminator_out_gen_in)
    gen_bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(gt_label,discriminator_out_gen_in) #y_true, y_pred
    gen_total_loss = gen_bce_loss + LAMBDA*gen_L1_loss

    return gen_total_loss,gen_bce_loss,gen_L1_loss

def discriminator_loss(disc_real_output, disc_generated_output):
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss
    #total_disc_loss = 0.5*(real_loss + generated_loss) # from paper

    return total_disc_loss

In [ ]:
discriminator = discriminator_model(outlogit_no_sigmoid=True)
tf.keras.utils.plot_model(discriminator, to_file='./dis_model.png',show_shapes=True, dpi=64)

In [ ]:
generator = generator_model()
tf.keras.utils.plot_model(generator, to_file='./gen_model.png',show_shapes=True, dpi=64)


In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

@tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    #get generator out -
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)


def fit(train_ds, epochs, test_ds):
  for epoch in range(epochs):
    start = time.time()
    display.clear_output(wait=True)
    #plt.clf()

    for example_input, example_target in test_ds.take(1):
      generate_images(generator, example_input, example_target)#epoch=epoch)
    print("Epoch: ", epoch)

    # Train
    for n, (input_image, target) in train_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image, target, epoch)
    print()

    # saving (checkpoint) the model every 10 epochs
    if (epoch + 1) % 5 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

**Load Training Data**

In [ ]:
from matplotlib import pyplot as plt
from IPython import display
%matplotlib inline

In [ ]:
BUFFER_SIZE = 400
BATCH_SIZE = 16
EPOCHS = 75
IMG_WIDTH = 256
IMG_HEIGHT = 256
PLOT = False
PLOT_JITTER = False
do_test = False
import sys
from sklearn.model_selection import train_test_split
pathdata = r'./gan_pair_list_updated.npy'
data = np.load(pathdata,allow_pickle=True)
data = np.array(data)
train_files, test_files, = train_test_split(data,test_size=0.20,
                                    shuffle=True,random_state=42)
BUFFER_SIZE_TRAIN = train_files.shape[0]
BUFFER_SIZE_TEST = test_files.shape[0]
print(data.shape,train_files.shape,test_files.shape)

if True:
    train_dataset = tf.data.Dataset.from_tensor_slices(train_files)
    train_dataset = train_dataset.map(load_image_train,
                                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_dataset = train_dataset.shuffle(400)
    train_dataset = train_dataset.batch(1)
    #(img1,img2) = train_dataset.take(1)
    
    for input_image, target in train_dataset.as_numpy_iterator():
        plt.subplot(121)
        plt.imshow(np.squeeze(input_image))
        plt.subplot(122)
        plt.imshow(np.squeeze(target))
        plt.show()
        break


**Define generators**

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_files)
#train_dataset = tf.data.Dataset.list_files(os.path.join(PATH , 'train/*.jpg'))
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE_TRAIN)
train_dataset = train_dataset.batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices(test_files)
#test_dataset = tf.data.Dataset.list_files(os.path.join(PATH , 'test/*.jpg'))
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)



generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

checkpoint_dir = './kd_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


log_dir = "logs/"
summary_writer = tf.summary.create_file_writer(
    log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

**Training Loop**

In [ ]:

for example_input, example_target in test_dataset.take(1):
    generate_images(generator, example_input, example_target)

if True:
    try:
        fit(train_dataset, EPOCHS, test_dataset)
    except Exception as E:
        print(E)

In [ ]:
if True:
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    # Run the trained model on a few examples from the test dataset
    cont =0
    for inp, tar in test_dataset.take(25):
        generate_images(generator, inp, tar)
        cont +=1